In [126]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import sparse
from scipy.sparse import linalg

In [127]:
l = 1
dt = 0.01
omega = 1./15
eta = 0.03
A_0 = 1./30
# static hotspots
theta = 0.56
Gamma = 0.019
# dynamic hotspots
#theta = 5.6
#Gamma = 0.002
z = 2.

D = l**2 / dt
epsilon = theta*dt
gamma = Gamma / l**2
B_bar = epsilon*D*gamma / omega

A_init = A_0 + B_bar
rho_init = B_bar / A_init


In [128]:
N_spatial = 512
x = np.arange(0, N_spatial, l)

In [129]:
def laplacian(vec):
    r = np.arange(len(vec))
    lshift = (r + 1) % len(vec)
    rshift = (r - 1) % len(vec)
    
    return (vec[rshift] + vec[lshift] - 2*vec) / l**2

def grad(vec):
    r = np.arange(len(vec))
    lshift = (r + 1) % len(vec)
    rshift = (r - 1) % len(vec)
    
    return (vec[lshift] - vec[rshift]) / 2*l

In [130]:
def solve_B(B_n, A_n, rho_n, return_mats=False):
    diag = 1 + omega*dt - eta*D*dt / z * -2/l**2
    off_diag = eta*D*dt/z * -1/l**2
    
    diag_vec = diag*np.ones(len(A_n))
    off_diag_vec = off_diag*np.ones(len(A_n))
    
    lan = len(A_n) - 1
    lhs_mat = sparse.diags([diag_vec, off_diag_vec, off_diag_vec, off_diag_vec, off_diag_vec], [0, -1, 1, lan, -lan], format='csc')
    
    rhs_vec = B_n + epsilon*D*dt*rho_n*A_n
    
    res = linalg.spsolve(lhs_mat, rhs_vec)
    if return_mats:
        return lhs_mat, rhs_vec, res
    return res

In [131]:
def solve_rho(B_n, A_n, rho_n, return_mats=False):
    laplacian_A = laplacian(A_n)
    grad_A = grad(A_n)
    grad_rho = grad(rho_n)
    f = 2*D*dt / z * (laplacian_A / A_n - grad_A**2 / A_n**2) + A_n*dt
    C = np.max(f)
    #C = 0
    
    diag = 1 + C - D*dt/z * -2/l**2
    off_diag = -D*dt/z * 1/l**2
    
    diag_vec = diag*np.ones(len(A_n))
    off_diag_vec = off_diag*np.ones(len(A_n))
    
    lan = len(A_n) - 1
    lhs_mat = sparse.diags([diag_vec, off_diag_vec, off_diag_vec, off_diag_vec, off_diag_vec], [0, -1, 1, lan, -lan], format='csc')
    rhs_vec = (1 + C - f) * rho_n - 2*D*dt/z * grad_rho*grad_A / A_n + gamma*dt
    
    res = linalg.spsolve(lhs_mat, rhs_vec)
    if return_mats:
        return lhs_mat, rhs_vec, res
    return res

In [132]:
def solve_forward(times):
    A = []
    rho = []

    A_n = A_init*np.ones(len(x)) + np.random.normal(0, 0.01, len(x))
    #A_n = A_init*np.ones(N_spatial)
    #for i in np.arange(0, len(A_n), 100):
    #    A_n[i] = A_n[i] * 1.1
    rho_n = rho_init*np.ones(len(x))

    A.append(A_n)
    rho.append(rho_n)
    for ix, t in enumerate(times):
        if ix % 100 == 0: print ix,
        B_next = solve_B(A_n-A_0, A_n, rho_n)
        A_next = B_next + A_0

        rho_next = solve_rho(A_n-A_0, A_n, rho_n)

        A.append(A_next)
        rho.append(rho_next)

        A_n = A_next
        rho_n = rho_next
    return A, rho

In [140]:
%%time
Tend = 500
A, rho = solve_forward(np.arange(dt, Tend+dt, dt))

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 8100 8200 8300 8400 8500 8600 8700 8800 8900 9000 9100 9200 9300 9400 9500 9600 9700 9800 9900 10000 10100 10200 10300 10400 10500 10600 10700 10800 10900 11000 11100 11200 11300 11400 11500 11600 11700 11800 11900 12000 12100 12200 12300 12400 12500 12600 12700 12800 12900 13000 13100 13200 13300 13400 13500 13600 13700 13800 13900 14000 14100 14200 14300 14400 14500 14600 14700 14800 14900 15000 15100 15200 15300 15400 15500 15600 15700 15800 15900 16000 16100 16200 16300 16400 16500 16600 16700 16800 16900 17000 17100 17200 17300 17400 17500 17600 17700 17800 17900 18000 18100 18200 18300 18400 18

In [141]:
from tempfile import NamedTemporaryFile

VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)
from matplotlib import animation
animation.Animation._repr_html_ = anim_to_html

In [142]:
# First set up the figure, the axis, and the plot element we want to animate
plt.figure(figsize=(10,10))
ax = plt.gca()
ax.set_xlim(0, N_spatial)
ax.set_ylim(0,1)
fig = plt.gcf()
A_line, = ax.plot([], [], label='A')
rho_line, = ax.plot([], [], label='rho')
skip = 100
plt.legend()
# initialization function: plot the background of each frame
def init():
    return []

# animation function.  This is called sequentially
def animate(i):
    A_line.set_data(x, A[skip*i])
    rho_line.set_data(x, rho[skip*i])
    return A_line, rho_line

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=len(A) / skip, interval=20, blit=True)
plt.close()
anim